In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## 1. Imports

In [2]:
import os
import datetime

!pip3 install ta
import ta
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(98)

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

from sklearn import metrics
from sklearn.preprocessing import StandardScaler

## 2. Configuration

In [3]:
prediction_periods = [2, 3, 5, 7, 10]
train_test_split_factors = [0.8, 0.75, 0.8, 0.75, 0.8]
activation = 'relu'
optimizer = 'adam'
loss = 'mse'
epochs = [10, 25, 20, 25, 20]
hidden_layer_configs = [(8, 32, 8), (100, 200, 100), (12, 32, 32, 12), (25, 75, 100, 75, 25), (100, 64, 64)]

## 3. Automated model generation

In [ ]:
companies_data = os.listdir('./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Storage/')

# For each company, generate models according to differrent configurations
for company_csv in companies_data:
  company_df = pd.read_csv(f'./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Storage/{company_csv}')
  
  # Calcuate returns according to prediction periods in config
  for prediction_period in prediction_periods:
    company_df[f'returns_period_{prediction_period}'] = list(company_df['close'][prediction_period:]) + [None for i in range(prediction_period)]
    company_df[f'returns_period_{prediction_period}'] = ((company_df[f'returns_period_{prediction_period}'] - company_df['close']) / company_df['close'])*100
  company_df.dropna(inplace=True)

  # Create 5 models per company
  for i in range(5):

    prediction_period = prediction_periods[i]

    # Split into test and train sets
    msk = np.random.rand(len(company_df)) < train_test_split_factors[i]
    train_df = company_df.copy()[msk]
    test_df = company_df.copy()[~msk]

    # Train set
    X_train = train_df.copy()
    X_train.drop(list(X_train.filter(regex='returns'))+['time_stamp'], axis=1, inplace=True)
    y_train = train_df.copy()[f'returns_period_{prediction_period}']

    # Test set
    X_test = test_df.copy()
    X_test.drop(list(X_test.filter(regex='returns'))+['time_stamp'], axis=1, inplace=True)
    y_test = test_df.copy()[f'returns_period_{prediction_period}']

    # Add indicators
    X_train = ta.add_all_ta_features(X_train, open="open", high="high", low="low", close="close", volume="volume", fillna=True)
    X_test = ta.add_all_ta_features(X_test, open="open", high="high", low="low", close="close", volume="volume", fillna=True)

    # Scale and Reshape datasets
    
    # Set scaler
    scaler = StandardScaler()

    # Scale and Reshape X_train
    X_train_standard_scaled = X_train.values
    X_train_standard_scaled = scaler.fit_transform(X_train_standard_scaled)
    X_train_standard_scaled = X_train_standard_scaled.reshape(X_train_standard_scaled.shape[0], X_train_standard_scaled.shape[1], 1)

    # Scale and Reshape X_test
    X_test_standard_scaled = X_test.values
    X_test_standard_scaled = scaler.fit_transform(X_test_standard_scaled)
    X_test_standard_scaled = X_test_standard_scaled.reshape(X_test_standard_scaled.shape[0], X_test_standard_scaled.shape[1], 1)

    # Scale and Reshape y_train
    y_train_standard_scaled = y_train.values
    y_train_standard_scaled = y_train_standard_scaled.reshape(y_train_standard_scaled.shape[0], 1)
    y_train_standard_scaled = scaler.fit_transform(y_train_standard_scaled)

    # Scale and Reshape y_test
    y_test_standard_scaled = y_test.values
    y_test_standard_scaled = y_test_standard_scaled.reshape(y_test_standard_scaled.shape[0], 1)
    y_test_standard_scaled = scaler.fit_transform(y_test_standard_scaled)

    # Define model
    model = Sequential()
    model.add(LSTM(hidden_layer_configs[i][0], activation=activation, return_sequences=True, input_shape=(89, 1)))
    for layer in range(1, len(hidden_layer_configs[i]) - 1):
      model.add(LSTM(hidden_layer_configs[i][layer], activation=activation, return_sequences=True))
    model.add(LSTM(hidden_layer_configs[i][-1], activation=activation))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss=loss)

    # Show a summary of the model
    print()
    # model.summary()

    # String to save model
    save_string = f"./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Models/{company_csv[:-7]}/LSTM_period:{prediction_period}_splitfactor:{train_test_split_factors[i]}_epochs:{epochs[i]}.h5"

    # Train model
    checkpoint = ModelCheckpoint(save_string, monitor='loss', verbose=1, save_best_only=True, mode='auto', period=1)
    model.fit(X_train_standard_scaled, y_train_standard_scaled, epochs=epochs[i], batch_size=1, callbacks=[checkpoint])

    # Test model
    predictions = model.predict(X_test_standard_scaled)
    prediction_percents = scaler.inverse_transform(predictions)

    # For normalized return
    normalized_mae = metrics.mean_absolute_error(y_test_standard_scaled, predictions)
    normalized_mse = metrics.mean_squared_error(y_test_standard_scaled, predictions)
    normalized_rmse = np.sqrt(metrics.mean_squared_error(y_test_standard_scaled, predictions))

    print()
    print('ERROR VALUES FOR NORMALIZED RETURN PERCENTS')
    print('Mean Absolute Error:', normalized_mae)
    print('Mean Squared Error:', normalized_mse)
    print('Root Mean Squared Error:', normalized_rmse)

    # For actual return percents
    actual_mae = metrics.mean_absolute_error(y_test, prediction_percents)
    actual_mse = metrics.mean_squared_error(y_test, prediction_percents)
    actual_rmse = np.sqrt(metrics.mean_squared_error(y_test, prediction_percents))

    print()
    print('ERROR VALUES FOR ACTUAL RETURN PERCENTS')
    print('Mean Absolute Error:', actual_mae)
    print('Mean Squared Error:', actual_mse)
    print('Root Mean Squared Error:', actual_rmse)

    # Rename saved model with more info
    new_save_string = f"./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Models/{company_csv[:-7]}/LSTM_period:{prediction_period}_splitfactor:{train_test_split_factors[i]}_epochs:{epochs[i]}_nmae:{normalized_mae}_nmse:{normalized_mse}__nrmse:{normalized_rmse}__mae:{actual_mae}__mse:{actual_mse}__rmse:{actual_rmse}_.h5"
    os.rename(save_string, new_save_string)

    # Consider building just one model for now
    break

  # Consider just one company for now  
  # break


/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])



Epoch 1/10
797/797 [==============================] - 50s 36ms/step - loss: 1.0032

Epoch 00001: loss improved from inf to 1.00112, saving model to ./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Models/ASIANPAINT/LSTM_period:2_splitfactor:0.8_epochs:10.h5
Epoch 2/10
797/797 [==============================] - 29s 36ms/step - loss: 1.3126

Epoch 00002: loss did not improve from 1.00112
Epoch 3/10
797/797 [==============================] - 28s 36ms/step - loss: 0.9462

Epoch 00003: loss did not improve from 1.00112
Epoch 4/10
797/797 [==============================] - 28s 35ms/step - loss: 0.9441

Epoch 00004: loss improved from 1.00112 to 0.99800, saving model to ./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Models/ASIANPAINT/LSTM_period:2_splitfactor:0.8_epochs:10.h5
Epoch 5/10
797/797 [==============================] - 28s 35ms/step - loss: 45.7938

Epoch 00005: loss did not improve from 0.99800
Epoch 6/10
797/797 [==============================] - 27s 34ms/step - loss: 0.8700

E

/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])



Epoch 1/10
770/770 [==============================] - 31s 35ms/step - loss: 0.9618

Epoch 00001: loss improved from inf to 1.00252, saving model to ./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Models/ADANIPORTS/LSTM_period:2_splitfactor:0.8_epochs:10.h5
Epoch 2/10
770/770 [==============================] - 26s 34ms/step - loss: 1.0206

Epoch 00002: loss improved from 1.00252 to 1.00066, saving model to ./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Models/ADANIPORTS/LSTM_period:2_splitfactor:0.8_epochs:10.h5
Epoch 3/10
770/770 [==============================] - 28s 37ms/step - loss: 0.9889

Epoch 00003: loss did not improve from 1.00066
Epoch 4/10
770/770 [==============================] - 28s 37ms/step - loss: 1.0020

Epoch 00004: loss did not improve from 1.00066
Epoch 5/10
770/770 [==============================] - 29s 38ms/step - loss: 1.0291

Epoch 00005: loss improved from 1.00066 to 0.99957, saving model to ./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Models/ADANIPORT

/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


...

/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])



Epoch 1/10
790/790 [==============================] - 32s 36ms/step - loss: 0.9602

Epoch 00001: loss improved from inf to 1.00152, saving model to ./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Models/COALINDIA/LSTM_period:2_splitfactor:0.8_epochs:10.h5
Epoch 2/10
790/790 [==============================] - 27s 34ms/step - loss: 1.0298

Epoch 00002: loss improved from 1.00152 to 1.00125, saving model to ./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Models/COALINDIA/LSTM_period:2_splitfactor:0.8_epochs:10.h5
Epoch 3/10
790/790 [==============================] - 29s 36ms/step - loss: 1.0042

Epoch 00003: loss improved from 1.00125 to 1.00093, saving model to ./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Models/COALINDIA/LSTM_period:2_splitfactor:0.8_epochs:10.h5
Epoch 4/10
790/790 [==============================] - 28s 35ms/step - loss: 0.8855

Epoch 00004: loss improved from 1.00093 to 1.00089, saving model to ./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Models/COALINDIA/LS

/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])



Epoch 1/10
797/797 [==============================] - 32s 36ms/step - loss: 0.9086

Epoch 00001: loss improved from inf to 1.00760, saving model to ./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Models/DIVISLAB/LSTM_period:2_splitfactor:0.8_epochs:10.h5
Epoch 2/10
797/797 [==============================] - 28s 35ms/step - loss: 0.9542

Epoch 00002: loss improved from 1.00760 to 1.00279, saving model to ./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Models/DIVISLAB/LSTM_period:2_splitfactor:0.8_epochs:10.h5
Epoch 3/10
797/797 [==============================] - 27s 34ms/step - loss: 0.8846

Epoch 00003: loss improved from 1.00279 to 1.00252, saving model to ./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Models/DIVISLAB/LSTM_period:2_splitfactor:0.8_epochs:10.h5
Epoch 4/10
797/797 [==============================] - 29s 36ms/step - loss: 0.9469

Epoch 00004: loss improved from 1.00252 to 1.00111, saving model to ./drive/MyDrive/Colab Notebooks/CAPSTONE PROJECT/Models/DIVISLAB/LSTM_p

ValueError: ignored